In [28]:
import torch
import numpy as np
from torchmetrics.retrieval import RetrievalNormalizedDCG

In [2]:
K=10

In [71]:
truth = torch.rand(2, 20)
truth = []
for i in range(2):
    a = torch.arange(20)
    a = a[torch.randperm(a.size()[0])]
    truth.append(a)
truth = torch.stack(truth)
preds = torch.rand(2, 20)

print(preds)
print(truth)

tensor([[0.1807, 0.9279, 0.1345, 0.6650, 0.6506, 0.9649, 0.6762, 0.4907, 0.6765,
         0.4774, 0.8684, 0.8787, 0.1453, 0.0715, 0.3327, 0.4553, 0.9419, 0.3236,
         0.2591, 0.9133],
        [0.0015, 0.9148, 0.3128, 0.2203, 0.3517, 0.3101, 0.0857, 0.8655, 0.8868,
         0.7008, 0.8773, 0.2196, 0.2142, 0.2294, 0.4055, 0.3190, 0.8570, 0.3766,
         0.9754, 0.4491]])
tensor([[10, 16,  1,  6,  4,  8,  7, 13,  2, 15, 11,  5, 14,  0, 12, 17, 18,  9,
         19,  3],
        [18,  0,  5, 11, 19, 15, 16, 17,  4, 10,  6,  9, 14, 12,  8,  1, 13,  2,
          3,  7]])


In [72]:
top_k_indices = torch.topk(preds, k=K)[1]
top_k_indices

tensor([[ 5, 16,  1, 19, 11, 10,  8,  6,  3,  4],
        [18,  1,  8, 10,  7, 16,  9, 19, 14, 17]])

In [73]:
top_k_truths = torch.gather(truth, 1, top_k_indices)
top_k_truths

tensor([[ 8, 18, 16,  3,  5, 11,  2,  7,  6,  4],
        [ 3,  0,  4,  6, 17, 13, 10,  7,  8,  2]])

In [74]:
ideal_truths = torch.sort(top_k_truths, dim=1, descending=True)[0]
ideal_truths

tensor([[18, 16, 11,  8,  7,  6,  5,  4,  3,  2],
        [17, 13, 10,  8,  7,  6,  4,  3,  2,  0]])

In [75]:
def dcg(ctruths):
    dcg = ctruths / torch.log2(torch.arange(ctruths.size(-1)) + 2)
    return dcg.sum(dim=1)

In [76]:
ndcg = dcg(top_k_truths) / dcg(ideal_truths)
# ndcg = ndcg.mean()
ndcg

tensor([0.8713, 0.6603])

# Verification

In [77]:
def callback(a, dim):
    return a

In [78]:
ndcg = RetrievalNormalizedDCG(top_k=K, aggregation=callback)

indexes = torch.tensor([[i] * truth.size(1) for i in range(truth.size(0))])
ndcg(preds.flatten(), truth.flatten(), indexes=indexes.flatten())

tensor([0.5693, 0.3855])